In [1]:
import SimpleITK as sitk
import matplotlib.pylab as plt
import os
import csv
import numpy as np
import imageio
from scipy import ndimage
import cv2

In [2]:
def autocropmin(image, threshold=0, kernsel_size = 10):
        
    img = image.copy()
    SIZE = img.shape[0]
    imgfilt = ndimage.minimum_filter(img.max((2)), size=kernsel_size)
    rows = np.where(np.max(imgfilt, 0) > threshold)[0]
    cols = np.where(np.max(imgfilt, 1) > threshold)[0]
    row1, row2 = rows[0], rows[-1] + 1
    col1, col2 = cols[0], cols[-1] + 1
    row1, col1 = max(0, row1-kernsel_size), max(0, col1-kernsel_size)
    row2, col2 = min(SIZE, row2+kernsel_size), min(SIZE, col2+kernsel_size)
    image = image[col1: col2, row1: row2]
    #logger.info(image.shape)
    sqside = max(image.shape)
    imageout = np.zeros((sqside, sqside, 3), dtype = 'uint8')
    imageout[:image.shape[0], :image.shape[1],:] = image.copy()
    return imageout

In [3]:
path = 'image_dong/Ori_Image/NC'

In [4]:
lipath = 'image_dong/Liver_Mask/NC'

In [5]:
tupath = 'Tumor_for_one/NC'

In [6]:
def file_name(file_dir): 
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.mhd':
                L.append(os.path.join(root, file))
    return L

In [7]:
def file_name_t(file_dir): 
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.npy':
                L.append(os.path.join(root, file))
    return L

In [8]:
l = file_name(path)
li = file_name(lipath)
lt = file_name_t(tupath)

In [9]:
def image_name(file_dir):
    L=[]
    for i in file_dir:
        L.append(i[24:30])
    L = list(set(L))
    L.sort()
    return L

In [10]:
def image_name_i(file_dir):
    L=[]
    for i in file_dir:
        L.append(i[25:31])
    L = list(set(L))
    L.sort()
    return L

In [11]:
def image_name_t(file_dir):
    L=[]
    for i in file_dir:
        L.append(i[17:23])
    L = list(set(L))
    L.sort()
    return L

In [12]:
n = image_name(l)
ni = image_name_i(li)
nt = image_name_t(lt)

In [14]:
cout = [0]*5
for i in n:
    if i[3] == '1':
        cout[0]+=1
    if i[3] == '2':
        cout[1]+=1
    if i[3] == '3':
        cout[2]+=1
    if i[3] == '4':
        cout[3]+=1
    if i[3] == '5':
        cout[4]+=1

In [15]:
print(cout)

[30, 19, 20, 20, 11]


In [16]:
print(len(n))
print(len(ni))
print(len(nt))

100
119
117


In [17]:
from sklearn.model_selection import train_test_split

In [18]:
n1 = []
n2 = []
n3 = []
n4 = []
n5 = []
cout = [0]*5
for i in n:
    if i[3] == '1':
        n1.append(i)
    if i[3] == '2':
        n2.append(i)
    if i[3] == '3':
        n3.append(i)
    if i[3] == '4':
        n4.append(i)
    if i[3] == '5':
        n5.append(i)

In [19]:
train1 , test1 = train_test_split(n1,test_size = 0.3333)
train2 , test2 = train_test_split(n2,test_size = 0.3333)
train3 , test3 = train_test_split(n3,test_size = 0.3333)
train4 , test4 = train_test_split(n4,test_size = 0.3333)
train5 , test5 = train_test_split(n5,test_size = 0.3333)

train1 , val1 = train_test_split(train1,test_size = 0.5)
train2 , val2 = train_test_split(train2,test_size = 0.5)
train3 , val3 = train_test_split(train3,test_size = 0.5)
train4 , val4 = train_test_split(train4,test_size = 0.5)
train5 , val5 = train_test_split(train5,test_size = 0.5)

In [20]:
train = train1+train2+train3+train4+train5
test = test1+test2+test3+test4+test5
val = val1+val2+val3+val4+val5

In [21]:
cout = [0]*5
for i in train:
    if i[3] == '1':
        cout[0]+=1
    if i[3] == '2':
        cout[1]+=1
    if i[3] == '3':
        cout[2]+=1
    if i[3] == '4':
        cout[3]+=1
    if i[3] == '5':
        cout[4]+=1
print(cout)
cout = [0]*5
for i in test:
    if i[3] == '1':
        cout[0]+=1
    if i[3] == '2':
        cout[1]+=1
    if i[3] == '3':
        cout[2]+=1
    if i[3] == '4':
        cout[3]+=1
    if i[3] == '5':
        cout[4]+=1
print(cout)
cout = [0]*5
for i in val:
    if i[3] == '1':
        cout[0]+=1
    if i[3] == '2':
        cout[1]+=1
    if i[3] == '3':
        cout[2]+=1
    if i[3] == '4':
        cout[3]+=1
    if i[3] == '5':
        cout[4]+=1
print(cout)

[10, 6, 6, 6, 3]
[10, 7, 7, 7, 4]
[10, 6, 7, 7, 4]


In [22]:
def norm_liver(x):
    x[x<-45] = -45
    x[x>105] = 105
    return (255*(x-np.min(x))/(np.max(x)-np.min(x))).astype(np.uint8)

In [23]:
def norm_nan(x):
    x[x<-150] = -150
    x[x>250] = 250
    return (255*(x-np.min(x))/(np.max(x)-np.min(x))).astype(np.uint8)

In [24]:
def norm_tsu(x):
    x[x<-500] = -500
    x[x>1300] = 1300
    return (255*(x-np.min(x))/(np.max(x)-np.min(x))).astype(np.uint8)

In [25]:
for i in train:
    ct_scans_NC = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Ori_Image/NC'+"/"+i+"_NC.mhd", sitk.sitkFloat32))
    ct_scans_ART = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Ori_Image/ART'+"/"+i+"_ART.mhd", sitk.sitkFloat32))
    ct_scans_PV = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Ori_Image/PV'+"/"+i+"_PV.mhd", sitk.sitkFloat32))
    ct_scans_l_NC = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Liver_Mask/NC'+"/"+i+"_NC_LiverMask.mhd", sitk.sitkFloat32))
    ct_scans_l_ART = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Liver_Mask/ART'+"/"+i+"_ART_LiverMask.mhd", sitk.sitkFloat32))
    ct_scans_l_PV = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Liver_Mask/PV'+"/"+i+"_PV_LiverMask.mhd", sitk.sitkFloat32))
    sliceb = np.load('./mean_num/'+i+'.npy')

    if not os.path.exists('./'+'Tumor_for_one/NC'+"/"+i+'.npy'):
        print('./'+'Tumor_for_one/NC'+"/"+i+'.npy')
        continue
    else:
        print('./'+'Tumor_for_one/NC'+"/"+i+'.npy')
        ct_scans_t_nc = np.load('./'+'Tumor_for_one/NC'+"/"+i+'.npy')
        
    if not os.path.exists('./'+'Tumor_for_one/ART'+"/"+i+'.npy'):
        continue
    else:     
        ct_scans_t_art = np.load('./'+'Tumor_for_one/ART'+"/"+i+'.npy')
        
    if not os.path.exists('./'+'Tumor_for_one/PV'+"/"+i+'.npy'):
        continue
    else:     
        ct_scans_t_pv = np.load('./'+'Tumor_for_one/PV'+"/"+i+'.npy')
    
    x,y,z = int(sliceb[0]),int(sliceb[1]),int(sliceb[2])
    if not os.path.exists('./3fold/train/'+str(int(i[3])-1)+'/'+i):
        os.mkdir('./3fold/train/'+str(int(i[3])-1)+'/'+i)
            
    for j in range(-10,10):
        if x+j>=ct_scans_NC.shape[0] or y+j>=ct_scans_ART.shape[0] or z+j>=ct_scans_PV.shape[0]:
            continue
        x_nc = norm_liver(ct_scans_NC[x+j])
        x_art = norm_liver(ct_scans_ART[y+j])
        x_pv = norm_liver(ct_scans_PV[z+j])
        y_nc = ct_scans_l_NC[x+j]
        y_art = ct_scans_l_ART[y+j]
        y_pv = ct_scans_l_PV[z+j]
        z_nc = ct_scans_t_nc[x+j]
        z_art = ct_scans_t_art[y+j]
        z_pv = ct_scans_t_pv[z+j]
        if np.sum(z_nc) == 0 and np.sum(z_art) == 0 and np.sum(z_pv) == 0:
            print('negative slice')
        elif np.sum(z_nc) >= 200 and np.sum(z_art) >= 200 and np.sum(z_pv) >= 200:
            #if np.sum(y_nc) >= 200 and np.sum(y_art) >= 200 and np.sum(y_pv) >= 200:
            #    imageio.imwrite('./3fold/train/'+str(int(i[3])-1)+'/'+i+'/'+i+'_'+str(j+4)+'.png', autocropmin(cv2.merge([x_nc*y_nc,x_art*y_art,x_pv*y_pv])))
            #else:
            imageio.imwrite('./3fold/train/'+str(int(i[3])-1)+'/'+i+'/'+i+'_'+str(j+4)+'.png', autocropmin(cv2.merge([x_nc*z_nc,x_art*z_art,x_pv*z_pv])))

./Tumor_for_one/NC/Srr107.npy
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
./Tumor_for_one/NC/Srr112.npy
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
./Tumor_for_one/NC/Srr119.npy
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
./Tumor_for_one/NC/Srr109.npy
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative s

In [26]:
for i in test:
    ct_scans_NC = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Ori_Image/NC'+"/"+i+"_NC.mhd", sitk.sitkFloat32))
    ct_scans_ART = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Ori_Image/ART'+"/"+i+"_ART.mhd", sitk.sitkFloat32))
    ct_scans_PV = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Ori_Image/PV'+"/"+i+"_PV.mhd", sitk.sitkFloat32))
    ct_scans_l_NC = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Liver_Mask/NC'+"/"+i+"_NC_LiverMask.mhd", sitk.sitkFloat32))
    ct_scans_l_ART = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Liver_Mask/ART'+"/"+i+"_ART_LiverMask.mhd", sitk.sitkFloat32))
    ct_scans_l_PV = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Liver_Mask/PV'+"/"+i+"_PV_LiverMask.mhd", sitk.sitkFloat32))
    sliceb = np.load('./mean_num/'+i+'.npy')

    if not os.path.exists('./'+'Tumor_for_one/NC'+"/"+i+'.npy'):
        print('./'+'Tumor_for_one/NC'+"/"+i+'.npy')
        continue
    else:
        print('./'+'Tumor_for_one/NC'+"/"+i+'.npy')
        ct_scans_t_nc = np.load('./'+'Tumor_for_one/NC'+"/"+i+'.npy')
        
    if not os.path.exists('./'+'Tumor_for_one/ART'+"/"+i+'.npy'):
        continue
    else:     
        ct_scans_t_art = np.load('./'+'Tumor_for_one/ART'+"/"+i+'.npy')
        
    if not os.path.exists('./'+'Tumor_for_one/PV'+"/"+i+'.npy'):
        continue
    else:     
        ct_scans_t_pv = np.load('./'+'Tumor_for_one/PV'+"/"+i+'.npy')
    
    x,y,z = int(sliceb[0]),int(sliceb[1]),int(sliceb[2])
    if not os.path.exists('./3fold/test/'+str(int(i[3])-1)+'/'+i):
        os.mkdir('./3fold/test/'+str(int(i[3])-1)+'/'+i)
            
    for j in range(-10,10):
        if x+j>=ct_scans_NC.shape[0] or y+j>=ct_scans_ART.shape[0] or z+j>=ct_scans_PV.shape[0]:
            continue
        x_nc = norm_liver(ct_scans_NC[x+j])
        x_art = norm_liver(ct_scans_ART[y+j])
        x_pv = norm_liver(ct_scans_PV[z+j])
        y_nc = ct_scans_l_NC[x+j]
        y_art = ct_scans_l_ART[y+j]
        y_pv = ct_scans_l_PV[z+j]
        z_nc = ct_scans_t_nc[x+j]
        z_art = ct_scans_t_art[y+j]
        z_pv = ct_scans_t_pv[z+j]
        if np.sum(z_nc) == 0 and np.sum(z_art) == 0 and np.sum(z_pv) == 0:
            print('negative slice')
        elif np.sum(z_nc) >= 200 and np.sum(z_art) >= 200 and np.sum(z_pv) >= 200:
            #if np.sum(y_nc) >= 200 and np.sum(y_art) >= 200 and np.sum(y_pv) >= 200:
            #    imageio.imwrite('./3fold/test/'+str(int(i[3])-1)+'/'+i+'/'+i+'_'+str(j+4)+'.png', autocropmin(cv2.merge([x_nc*y_nc,x_art*y_art,x_pv*y_pv])))
            #else:
            imageio.imwrite('./3fold/test/'+str(int(i[3])-1)+'/'+i+'/'+i+'_'+str(j+4)+'.png', autocropmin(cv2.merge([x_nc*z_nc,x_art*z_art,x_pv*z_pv])))

./Tumor_for_one/NC/Srr135.npy
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
./Tumor_for_one/NC/Srr101.npy
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
./Tumor_for_one/NC/Srr133.npy
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
./Tumor_for_one/NC/Srr117.npy
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative s

In [27]:
for i in val:
    ct_scans_NC = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Ori_Image/NC'+"/"+i+"_NC.mhd", sitk.sitkFloat32))
    ct_scans_ART = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Ori_Image/ART'+"/"+i+"_ART.mhd", sitk.sitkFloat32))
    ct_scans_PV = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Ori_Image/PV'+"/"+i+"_PV.mhd", sitk.sitkFloat32))
    ct_scans_l_NC = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Liver_Mask/NC'+"/"+i+"_NC_LiverMask.mhd", sitk.sitkFloat32))
    ct_scans_l_ART = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Liver_Mask/ART'+"/"+i+"_ART_LiverMask.mhd", sitk.sitkFloat32))
    ct_scans_l_PV = sitk.GetArrayFromImage(sitk.ReadImage("./"+'image_dong/Liver_Mask/PV'+"/"+i+"_PV_LiverMask.mhd", sitk.sitkFloat32))
    sliceb = np.load('./mean_num/'+i+'.npy')

    if not os.path.exists('./'+'Tumor_for_one/NC'+"/"+i+'.npy'):
        print('./'+'Tumor_for_one/NC'+"/"+i+'.npy')
        continue
    else:
        print('./'+'Tumor_for_one/NC'+"/"+i+'.npy')
        ct_scans_t_nc = np.load('./'+'Tumor_for_one/NC'+"/"+i+'.npy')
        
    if not os.path.exists('./'+'Tumor_for_one/ART'+"/"+i+'.npy'):
        continue
    else:     
        ct_scans_t_art = np.load('./'+'Tumor_for_one/ART'+"/"+i+'.npy')
        
    if not os.path.exists('./'+'Tumor_for_one/PV'+"/"+i+'.npy'):
        continue
    else:     
        ct_scans_t_pv = np.load('./'+'Tumor_for_one/PV'+"/"+i+'.npy')
    
    x,y,z = int(sliceb[0]),int(sliceb[1]),int(sliceb[2])
    if not os.path.exists('./3fold/val/'+str(int(i[3])-1)+'/'+i):
        os.mkdir('./3fold/val/'+str(int(i[3])-1)+'/'+i)
            
    for j in range(-10,10):
        if x+j>=ct_scans_NC.shape[0] or y+j>=ct_scans_ART.shape[0] or z+j>=ct_scans_PV.shape[0]:
            continue
        x_nc = norm_liver(ct_scans_NC[x+j])
        x_art = norm_liver(ct_scans_ART[y+j])
        x_pv = norm_liver(ct_scans_PV[z+j])
        y_nc = ct_scans_l_NC[x+j]
        y_art = ct_scans_l_ART[y+j]
        y_pv = ct_scans_l_PV[z+j]
        z_nc = ct_scans_t_nc[x+j]
        z_art = ct_scans_t_art[y+j]
        z_pv = ct_scans_t_pv[z+j]
        if np.sum(z_nc) == 0 and np.sum(z_art) == 0 and np.sum(z_pv) == 0:
            print('negative slice')
        elif np.sum(z_nc) >= 200 and np.sum(z_art) >= 200 and np.sum(z_pv) >= 200:
            #if np.sum(y_nc) >= 200 and np.sum(y_art) >= 200 and np.sum(y_pv) >= 200:
            #    imageio.imwrite('./3fold/val/'+str(int(i[3])-1)+'/'+i+'/'+i+'_'+str(j+4)+'.png', autocropmin(cv2.merge([x_nc*y_nc,x_art*y_art,x_pv*y_pv])))
            #else:
            imageio.imwrite('./3fold/val/'+str(int(i[3])-1)+'/'+i+'/'+i+'_'+str(j+4)+'.png', autocropmin(cv2.merge([x_nc*z_nc,x_art*z_art,x_pv*z_pv])))

./Tumor_for_one/NC/Srr102.npy
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
./Tumor_for_one/NC/Srr105.npy
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
./Tumor_for_one/NC/Srr108.npy
negative slice
negative slice
negative slice
negative slice
negative slice
./Tumor_for_one/NC/Srr110.npy
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
./Tumor_for_one/NC/Srr115.npy
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
negative slice
./Tumor_fo